In [9]:
import pandas as pd

In [10]:
df = pd.read_csv('data/amazon_reviews_us_Video_Games_v1_00.tsv', sep='\t', error_bad_lines=False)
df.head()

b'Skipping line 20630: expected 15 fields, saw 22\nSkipping line 28172: expected 15 fields, saw 22\nSkipping line 54791: expected 15 fields, saw 22\n'
b'Skipping line 75419: expected 15 fields, saw 22\nSkipping line 104832: expected 15 fields, saw 22\n'
b'Skipping line 138464: expected 15 fields, saw 22\nSkipping line 194849: expected 15 fields, saw 22\n'
b'Skipping line 201568: expected 15 fields, saw 22\nSkipping line 242567: expected 15 fields, saw 22\n'
b'Skipping line 493585: expected 15 fields, saw 22\nSkipping line 502478: expected 15 fields, saw 22\n'
b'Skipping line 660750: expected 15 fields, saw 22\n'


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,12039526,RTIS3L2M1F5SM,B001CXYMFS,737716809,Thrustmaster T-Flight Hotas X Flight Stick,Video Games,5,0,0,N,Y,an amazing joystick. I especially love that yo...,"Used this for Elite Dangerous on my mac, an am...",2015-08-31
1,US,9636577,R1ZV7R40OLHKD,B00M920ND6,569686175,Tonsee 6 buttons Wireless Optical Silent Gamin...,Video Games,5,0,0,N,Y,Definitely a silent mouse... Not a single clic...,"Loved it, I didn't even realise it was a gami...",2015-08-31
2,US,2331478,R3BH071QLH8QMC,B0029CSOD2,98937668,Hidden Mysteries: Titanic Secrets of the Fatef...,Video Games,1,0,1,N,Y,One Star,poor quality work and not as it is advertised.,2015-08-31
3,US,52495923,R127K9NTSXA2YH,B00GOOSV98,23143350,GelTabz Performance Thumb Grips - PlayStation ...,Video Games,3,0,0,N,Y,"good, but could be bettee","nice, but tend to slip away from stick in inte...",2015-08-31
4,US,14533949,R32ZWUXDJPW27Q,B00Y074JOM,821342511,Zero Suit Samus amiibo - Japan Import (Super S...,Video Games,4,0,0,N,Y,Great but flawed.,"Great amiibo, great for collecting. Quality ma...",2015-08-31


In [11]:
df = pd.concat([df, pd.read_csv('data/amazon_reviews_us_Digital_Video_Games_v1_00.tsv', sep='\t', error_bad_lines=False)])
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,12039526,RTIS3L2M1F5SM,B001CXYMFS,737716809,Thrustmaster T-Flight Hotas X Flight Stick,Video Games,5,0,0,N,Y,an amazing joystick. I especially love that yo...,"Used this for Elite Dangerous on my mac, an am...",2015-08-31
1,US,9636577,R1ZV7R40OLHKD,B00M920ND6,569686175,Tonsee 6 buttons Wireless Optical Silent Gamin...,Video Games,5,0,0,N,Y,Definitely a silent mouse... Not a single clic...,"Loved it, I didn't even realise it was a gami...",2015-08-31
2,US,2331478,R3BH071QLH8QMC,B0029CSOD2,98937668,Hidden Mysteries: Titanic Secrets of the Fatef...,Video Games,1,0,1,N,Y,One Star,poor quality work and not as it is advertised.,2015-08-31
3,US,52495923,R127K9NTSXA2YH,B00GOOSV98,23143350,GelTabz Performance Thumb Grips - PlayStation ...,Video Games,3,0,0,N,Y,"good, but could be bettee","nice, but tend to slip away from stick in inte...",2015-08-31
4,US,14533949,R32ZWUXDJPW27Q,B00Y074JOM,821342511,Zero Suit Samus amiibo - Japan Import (Super S...,Video Games,4,0,0,N,Y,Great but flawed.,"Great amiibo, great for collecting. Quality ma...",2015-08-31


In [12]:
len(df)

1924992

In [13]:
criteria = {'review_id': 'count', 
            'star_rating': ['mean', 'std'], 
            'total_votes': ['mean', 'std'],
            'helpful_votes': ['mean', 'std']}
customer_df = df.groupby(df.customer_id).agg(criteria).fillna(0)
product_df = df.groupby(df.product_id).agg(criteria).fillna(0)
lookup_table = df[['customer_id', 'product_id', 'star_rating']]

In [16]:
import numpy as np
swapped = np.swapaxes(customer_df,0,1)
swapped.shape

(7, 1125953)

In [34]:
import skfuzzy as fuzz

In [36]:
models = []
for num_clusters in range(2, 8):
    print('\n---', num_clusters, '---')
    cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(swapped, num_clusters, 2, 0.005, 1000)
    models.append((cntr, u, u0, d, jm, p, fpc))
    print('FPC:', fpc)
    cluster_membership = np.argmax(u, axis=0)
    for i  in range(num_clusters):
        cluster = cluster_membership[cluster_membership == i]
        print('Cluster {}: {}%'.format(i, (len(cluster) / len(cluster_membership)) * 100))


--- 2 ---
FPC: 0.9661281718166361
Cluster 0: 2.2099501488960906%
Cluster 1: 97.79004985110392%

--- 3 ---
FPC: 0.9419680282920196
Cluster 0: 4.651437493394484%
Cluster 1: 95.2986492331385%
Cluster 2: 0.049913273467009724%

--- 4 ---
FPC: 0.8996314574719949
Cluster 0: 90.6400178337817%
Cluster 1: 0.6568657839181565%
Cluster 2: 0.0164305259633395%
Cluster 3: 8.68668585633681%

--- 5 ---
FPC: 0.8725045217103989
Cluster 0: 0.0036413598080914564%
Cluster 1: 0.06589973116106977%
Cluster 2: 87.77977411135278%
Cluster 3: 10.79378979406778%
Cluster 4: 1.3568950036102752%

--- 6 ---
FPC: 0.8233155189158615
Cluster 0: 13.927490756718974%
Cluster 1: 2.9347583780140023%
Cluster 2: 0.026288841541343196%
Cluster 3: 82.765532841957%
Cluster 4: 0.3448634179224177%
Cluster 5: 0.0010657638462706702%

--- 7 ---
FPC: 0.774704689656206
Cluster 0: 0.8883141658666037%
Cluster 1: 0.10524417981922869%
Cluster 2: 0.0008881365385588919%
Cluster 3: 78.09224718971396%
Cluster 4: 4.56058112549991%
Cluster 5: 0.0115

In [41]:
cluster_pcts = np.swapaxes(models[5][1],0,1)
cluster_pcts.shape

(1125953, 7)

In [53]:
cluster_df = pd.DataFrame(cluster_pcts, index=customer_df.index, columns=['cluster_0', 
                                                                          'cluster_1', 
                                                                          'cluster_2',
                                                                          'cluster_3',
                                                                          'cluster_4', 
                                                                          'cluster_5',
                                                                          'cluster_6'])
cluster_df.head()

,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6
customer_id,,,,,,,
10018,0.001394,0.000108,3.086527e-07,0.822362,0.012592,0.000008,0.163536
10114,0.000278,0.000022,6.394714e-08,0.973026,0.002339,0.000002,0.024332
10146,0.000278,0.000022,6.147077e-08,0.964092,0.002517,0.000002,0.033090
10164,0.000245,0.000020,5.627820e-08,0.976589,0.002056,0.000001,0.021089
10192,0.000245,0.000020,5.627820e-08,0.976589,0.002056,0.000001,0.021089


In [54]:
customer_combined_df = pd.concat([customer_df, cluster_df], axis=1)
customer_combined_df

,"(review_id, count)","(star_rating, mean)","(star_rating, std)","(total_votes, mean)","(total_votes, std)","(helpful_votes, mean)","(helpful_votes, std)",cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6
customer_id,,,,,,,,,,,,,,
10018,4,4.25,0.500000,1.25,0.957427,0.25,0.500000,0.001394,0.000108,3.086527e-07,0.822362,0.012592,0.000008,0.163536
10114,2,5.00,0.000000,0.00,0.000000,0.00,0.000000,0.000278,0.000022,6.394714e-08,0.973026,0.002339,0.000002,0.024332
10146,1,5.00,0.000000,1.00,0.000000,1.00,0.000000,0.000278,0.000022,6.147077e-08,0.964092,0.002517,0.000002,0.033090
10164,1,5.00,0.000000,0.00,0.000000,0.00,0.000000,0.000245,0.000020,5.627820e-08,0.976589,0.002056,0.000001,0.021089
10192,1,5.00,0.000000,0.00,0.000000,0.00,0.000000,0.000245,0.000020,5.627820e-08,0.976589,0.002056,0.000001,0.021089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53096494,2,2.50,2.121320,14.00,7.071068,12.50,7.778175,0.022167,0.001107,2.726586e-06,0.130024,0.593656,0.000069,0.252973
53096501,1,5.00,0.000000,0.00,0.000000,0.00,0.000000,0.000245,0.000020,5.627820e-08,0.976589,0.002056,0.000001,0.021089
53096538,2,3.00,2.828427,2.50,2.121320,2.00,1.414214,0.002422,0.000180,5.037979e-07,0.505556,0.024798,0.000012,0.467031
